In [1]:
import numpy as np
import pandas as pd

import pickle

## Подготовка данных

In [2]:
players = pickle.load(open('./data/players.pkl', 'rb'))
results = pickle.load(open('./data/results.pkl', 'rb'))
tournaments = pickle.load(open('./data/tournaments.pkl', 'rb'))

Сделаем датафрейм с игроками

In [20]:
players = pd.DataFrame(players).T
players.head(5)

,id,name,patronymic,surname
1,1,Алексей,None,Абабилов
10,10,Игорь,,Абалов
11,11,Наталья,Юрьевна,Абалымова
12,12,Артур,Евгеньевич,Абальян
13,13,Эрик,Евгеньевич,Абальян


Отфильтруем результаты турниров, у которых нет поля mask

In [21]:
len(results)

5528

In [22]:
results = {k:v for k, v in results.items() if all(el.get('mask', None) is not None for el in v)}
len(results)

3890

In [54]:
for tournament_id in tournaments.id.values:
    if tournament_id in results:
        if len(results[tournament_id]) == 0:
            del results[tournament_id]

In [55]:
len(results)

3639

Построим датафрейм с турнирами

In [16]:
tournaments = pd.DataFrame(tournaments).T
tournaments.head(5)

,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
1,1,Чемпионат Южного Кавказа,2003-07-25T00:00:00+04:00,2003-07-27T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/1,[],None,None
2,2,Летние зори,2003-08-09T00:00:00+04:00,2003-08-09T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/1,[],None,None
3,3,Турнир в Ижевске,2003-11-22T00:00:00+03:00,2003-11-24T00:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None
4,4,Чемпионат Украины. Переходной этап,2003-10-11T00:00:00+04:00,2003-10-12T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None
5,5,Бостонское чаепитие,2003-10-10T00:00:00+04:00,2003-10-13T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None


Оставим только те, для которых есть результаты

In [50]:
tournaments = tournaments[tournaments.id.isin(results.keys())]
tournaments.head()

,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
22,22,Чемпионат России,2004-02-21T00:00:00+03:00,2004-02-22T00:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,"{'1': 15, '2': 15, '3': 15, '4': 15, '5': 15, ..."
76,76,Чемпионат России,2005-02-26T00:00:00+03:00,2005-02-27T00:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/3,[],None,"{'1': 15, '2': 15, '3': 15, '4': 15, '5': 15, ..."
141,141,Чемпионат России,2006-02-26T00:00:00+03:00,2006-02-26T00:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/4,[],None,"{'1': 15, '2': 15, '3': 15, '4': 15, '5': 15, ..."
226,226,Чемпионат России,2007-02-24T00:00:00+03:00,2007-02-25T00:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/6,[],None,"{'1': 15, '2': 15, '3': 15, '4': 15, '5': 15, ..."
315,315,Чемпионат России,2008-03-09T00:00:00+03:00,2008-03-10T00:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/7,[],None,"{'1': 15, '2': 15, '3': 15, '4': 15, '5': 15, ..."


И разобъем на train и test в зависимости от года

In [51]:
train_tournaments = tournaments[tournaments.dateStart.apply(lambda s: s[:4] == '2019')]
train_tournaments.head()

,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
4772,4772,Синхрон северных стран. Зимний выпуск,2019-01-05T19:00:00+03:00,2019-01-09T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 28379, 'name': 'Константин', 'patronym...",{'dateRequestsAllowedTo': '2019-01-09T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
4973,4973,Балтийский Берег. 3 игра,2019-01-25T19:05:00+03:00,2019-01-29T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 23030, 'name': 'Марина', 'patronymic':...",{'dateRequestsAllowedTo': '2019-01-28T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
4974,4974,Балтийский Берег. 4 игра,2019-03-01T19:05:00+03:00,2019-03-05T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 23030, 'name': 'Марина', 'patronymic':...",{'dateRequestsAllowedTo': '2019-03-04T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
4975,4975,Балтийский Берег. 5 игра,2019-04-05T19:05:00+03:00,2019-04-09T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 23030, 'name': 'Марина', 'patronymic':...",{'dateRequestsAllowedTo': '2019-04-08T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
4986,4986,ОВСЧ. 6 этап,2019-02-15T20:00:00+03:00,2019-02-19T20:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 59140, 'name': 'Борис', 'patronymic': ...",{'dateRequestsAllowedTo': '2019-02-19T23:59:59...,"{'1': 12, '2': 12, '3': 12}"


In [52]:
test_tournaments = tournaments[tournaments.dateStart.apply(lambda s: s[:4] == '2020')]
test_tournaments.head()

,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
4957,4957,Синхрон Биркиркары,2020-02-21T00:00:00+03:00,2020-02-27T23:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/53,"[{'id': 2421, 'name': 'Ася', 'patronymic': 'Се...",{'dateRequestsAllowedTo': '2020-02-27T18:00:00...,"{'1': 13, '2': 13, '3': 13}"
5414,5414,Синхрон северных стран,2020-01-03T19:00:00+03:00,2020-01-10T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/53,"[{'id': 28379, 'name': 'Константин', 'patronym...",{'dateRequestsAllowedTo': '2020-01-10T23:59:00...,"{'1': 12, '2': 12, '3': 12}"
5705,5705,Школьный Синхрон-lite. Выпуск 3.5,2020-01-01T00:05:00+03:00,2020-02-03T23:55:00+03:00,"{'id': 8, 'name': 'Асинхрон'}",/seasons/53,"[{'id': 23740, 'name': 'Владимир', 'patronymic...",{'dateRequestsAllowedTo': '2020-02-02T23:59:00...,"{'1': 12, '2': 12, '3': 12}"
5706,5706,(а)Синхрон-lite. Лига старта. Эпизод XI,2020-01-01T00:05:00+03:00,2020-02-03T23:55:00+03:00,"{'id': 8, 'name': 'Асинхрон'}",/seasons/53,"[{'id': 23740, 'name': 'Владимир', 'patronymic...",{'dateRequestsAllowedTo': '2020-02-02T23:59:00...,"{'1': 12, '2': 12, '3': 12}"
5707,5707,Школьный Синхрон-lite. Выпуск 3.6,2020-02-01T00:05:00+03:00,2020-03-09T23:55:00+03:00,"{'id': 8, 'name': 'Асинхрон'}",/seasons/53,"[{'id': 23740, 'name': 'Владимир', 'patronymic...",{'dateRequestsAllowedTo': '2020-03-06T23:59:00...,"{'1': 12, '2': 12, '3': 12}"


In [53]:
print(
    train_tournaments.shape[0],
    test_tournaments.shape[0],
    sep='\n',
)

671
169


## Baseline model

In [110]:
from collections import Counter


def count_correct_answers(string):
    return len([ch for ch in list(string) if ch not in ['X', '?', '0']])

def count_stats_by_player(results_dict):
    correct_answers_by_player = Counter()
    questions_by_player = Counter()
    tournaments_by_player = Counter()
    
    for key, value in results_dict.items():
        for team_info in value:
            n_correct_answers = count_correct_answers(team_info['mask'])
            n_questions = len(team_info['mask'])
            
            for players in team_info['teamMembers']:
                player_id = players['player']['id']
                
                correct_answers_by_player[player_id] += n_correct_answers
                questions_by_player[player_id] += n_questions
                tournaments_by_player[player_id] += 1
    
    return correct_answers_by_player, questions_by_player, tournaments_by_player

# for debug
correct_answers_by_player, questions_by_player, tournaments_by_player = count_stats_by_player(results)

Каждой паре (игрок, вопрос) поставим в соответствие следующий набор признаков:

1. Общая сила игрока (т.е. отношение числа отвеченных вопросов к числу заданных)
2. Сила игрока в конкретном турнире (отношение числа отвеченных вопросов к числу заданных, но в одном турнире)
3. Бинарная переменная - принял ли игрок участие больше чем в 10 турнирах.
4. Сложность вопроса (т.е. отношение числа команд, ответивших на вопрос, к общему числу команд)

Посчитаем общую силу игрока: отношение числа отвеченных вопросов к числу заданных.

In [133]:
player_strength = {}

for player_id in tournaments_by_player:
    player_strength[player_id] = correct_answers_by_player[player_id] / questions_by_player[player_id]

Посчитаем сложность вопроса

In [172]:
def count_question_complexity(results_dict):
    result = {}
    
    for key, value in results_dict.items():
        mask_len = len(value[0]['mask'])
        matrix = np.zeros((len(value), mask_len))
        
        for i in range(len(value)):
            try:
                matrix[i] = np.array(
                    list(
                        map(
                            int,
                            list(value[i]['mask'].rjust(mask_len, '0').replace('X', '0').replace('?', '0'))
                        )
                    )
                )
            except ValueError:
                # this means that some team has different length of mask
                # we just skip these tournaments
                print(key, 'has bad mask shapes')
                break
        
        complexitites = 1 - matrix.sum(axis=0) / matrix.shape[0]
        
        for idx, c in enumerate(complexitites):
            result[(key, idx)] = c
    
    return result

Сейчас для каждого вопроса есть его сложность:

In [173]:
count_question_complexity(results)

1780 has bad mask shapes
1967 has bad mask shapes
2087 has bad mask shapes
2201 has bad mask shapes
2512 has bad mask shapes
2611 has bad mask shapes
2733 has bad mask shapes
3071 has bad mask shapes
3154 has bad mask shapes
3162 has bad mask shapes
3487 has bad mask shapes
3619 has bad mask shapes
4043 has bad mask shapes
4123 has bad mask shapes
4135 has bad mask shapes
4560 has bad mask shapes
4889 has bad mask shapes
4953 has bad mask shapes
5585 has bad mask shapes
5586 has bad mask shapes
6026 has bad mask shapes


{(22, 0): 1.0,
 (22, 1): 0.43181818181818177,
 (22, 2): 0.18181818181818177,
 (22, 3): 0.34090909090909094,
 (22, 4): 0.6136363636363636,
 (22, 5): 0.20454545454545459,
 (22, 6): 0.06818181818181823,
 (22, 7): 0.7272727272727273,
 (22, 8): 1.0,
 (22, 9): 0.15909090909090906,
 (22, 10): 0.6136363636363636,
 (22, 11): 0.75,
 (22, 12): 0.13636363636363635,
 (22, 13): 0.4772727272727273,
 (22, 14): 0.43181818181818177,
 (22, 15): 0.6818181818181819,
 (22, 16): 1.0,
 (22, 17): 0.7045454545454546,
 (22, 18): 0.8181818181818181,
 (22, 19): 0.18181818181818177,
 (22, 20): 0.9090909090909091,
 (22, 21): 0.15909090909090906,
 (22, 22): 0.0,
 (22, 23): 0.6363636363636364,
 (22, 24): 0.5454545454545454,
 (22, 25): 0.7727272727272727,
 (22, 26): 0.09090909090909094,
 (22, 27): 0.0,
 (22, 28): 0.15909090909090906,
 (22, 29): 0.38636363636363635,
 (22, 30): 0.0,
 (22, 31): 0.045454545454545414,
 (22, 32): 0.40909090909090906,
 (22, 33): 0.15909090909090906,
 (22, 34): 0.25,
 (22, 35): 0.2954545454545

Посчитаем силу игрока в конкретном турнире:

In [177]:
def count_player_strength_in_tournament(results_dict):
    result = {}
    
    for key, value in results_dict.items():
        for team_info in value:
            n_correct_answers = team_info['questionsTotal']
            for player in team_info['teamMembers']:
                player_id = player['player']['id']
                result[(key, player_id)] = n_correct_answers / len(team_info['mask'])

    return result

In [178]:
count_player_strength_in_tournament(results)

{(22, 1560): 0.7444444444444445,
 (22, 2935): 0.7444444444444445,
 (22, 3270): 0.7444444444444445,
 (22, 4878): 0.7444444444444445,
 (22, 18935): 0.7444444444444445,
 (22, 32979): 0.7444444444444445,
 (22, 36497): 0.7444444444444445,
 (22, 707): 0.7222222222222222,
 (22, 13551): 0.7222222222222222,
 (22, 15442): 0.7222222222222222,
 (22, 25882): 0.7222222222222222,
 (22, 30475): 0.7222222222222222,
 (22, 34846): 0.7222222222222222,
 (22, 2694): 0.7222222222222222,
 (22, 20691): 0.7222222222222222,
 (22, 22482): 0.7222222222222222,
 (22, 22935): 0.7222222222222222,
 (22, 28513): 0.7222222222222222,
 (22, 29800): 0.7222222222222222,
 (22, 5526): 0.7111111111111111,
 (22, 14259): 0.7111111111111111,
 (22, 18686): 0.7111111111111111,
 (22, 20260): 0.7111111111111111,
 (22, 21805): 0.7111111111111111,
 (22, 29333): 0.7111111111111111,
 (22, 3207): 0.6888888888888889,
 (22, 3645): 0.6888888888888889,
 (22, 6212): 0.6888888888888889,
 (22, 8333): 0.6888888888888889,
 (22, 49151): 0.6888888888

Наконец, для каждого игрока выясним, участвовал ли он более чем в 10 турнирах

In [186]:
train_results ={}

for tournament_id in train_tournaments['id'].values:
    train_results[tournament_id] = results[tournament_id]
    
    
correct_answers_by_player, questions_by_player, tournaments_by_player = count_stats_by_player(train_results)

player_strength = {}
for player_id in tournaments_by_player:
    player_strength[player_id] = correct_answers_by_player[player_id] / questions_by_player[player_id]
    
question_complexity = count_question_complexity(train_results)
player_strength_in_tournament = count_player_strength_in_tournament(train_results)

6026 has bad mask shapes


In [190]:
for k, v in player_strength_in_tournament.items():
    print(k, v)
    break

(4772, 6212) 0.7777777777777778


In [196]:
data = []
for key, value in player_strength_in_tournament.items():
    tournament_id, player_id = key
    data.append(
        [
            player_id,
            tournament_id,
            player_strength[player_id],
            player_strength_in_tournament.get((tournament_id, player_id), 0),
            question_complexity.get((tournament_id, player_id), 1),
            int(tournaments_by_player[player_id] > 10),
        ]
    )

In [199]:
df = pd.DataFrame(data, columns=[
    'player_id',
    'tournament_id',
    'player_strength',
    'player_strength_in_tournament',
    'question_complexity',
    'has_played_more_than_10_times',
])

df

,player_id,tournament_id,player_strength,player_strength_in_tournament,question_complexity,has_played_more_than_10_times
0,6212,4772,0.706069,0.777778,1.0,1
1,18332,4772,0.730139,0.777778,1.0,1
2,18036,4772,0.725641,0.777778,1.0,1
3,22799,4772,0.665806,0.777778,1.0,1
4,15456,4772,0.626735,0.777778,1.0,1
...,...,...,...,...,...,...
451114,221567,6255,0.027778,0.027778,1.0,0
451115,217153,6255,0.000000,0.000000,1.0,0
451116,217154,6255,0.000000,0.000000,1.0,0
451117,217155,6255,0.000000,0.000000,1.0,0


Обучим на этих данных логистическую регрессию

In [201]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

lr.fit(df.drop(['player_id', 'tournament_id', 'player_strength'], axis=1), df['player_strength'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [202]:
lr.coef_

array([0.43825677, 0.01899904, 0.0766424 ])

In [203]:
from scipy.stats import kendalltau, spearmanr

In [ ]:
df.groupby('player_id')

In [205]:
lr.predict([[0.777778, 1.0, 1]])

array([0.60799634])

In [206]:
for dataframe in df.groupby('player_id'):
    print(df)
    break

        player_id  tournament_id  player_strength  \
0            6212           4772         0.706069   
1           18332           4772         0.730139   
2           18036           4772         0.725641   
3           22799           4772         0.665806   
4           15456           4772         0.626735   
...           ...            ...              ...   
451114     221567           6255         0.027778   
451115     217153           6255         0.000000   
451116     217154           6255         0.000000   
451117     217155           6255         0.000000   
451118     217156           6255         0.000000   

        player_strength_in_tournament  question_complexity  \
0                            0.777778                  1.0   
1                            0.777778                  1.0   
2                            0.777778                  1.0   
3                            0.777778                  1.0   
4                            0.777778                